In [32]:
import gradio as gr
import math
import random
import sympy

In [33]:
class Montgomery:
    def __init__(self,base,exponent,n):
        self.base = base
        self.exponent = exponent
        self.n = n
        
    @staticmethod
    def choose_r(n):
        # Ensure n is odd and greater than 3
        if n % 2 == 0 or n <= 3:
            raise ValueError("Modulus n must be odd and greater than 3.")

        # Find the smallest power of 2 greater than n
        r = 2
        while r <= n:
            r *= 2

        return r
    @staticmethod
    def extended_gcd(a, b):
        x0, x1, y0, y1 = 1, 0, 0, 1
        while b != 0:
            q, a, b = a // b, b, a % b
            x0, x1 = x1, x0 - q * x1
            y0, y1 = y1, y0 - q * y1
        return a, x0, y0
    
    @staticmethod
    def modinv(R, N):
        gcd, x, y =Montgomery.extended_gcd(R, N)
        if gcd != 1:
            raise ValueError(f"The modular inverse does not exist for {R} mod {N}.")
        return x % N

    @staticmethod
    def montgomery_multiply(a,b,n):
        r=Montgomery.choose_r(n)
        r_inv=Montgomery.modinv(r,n)
        n_inv=(r*(r_inv%n)-1)//n
        a_bar=(a*r)%n
        b_bar=(b*r)%n
        s=a_bar*b_bar
        t=(s*n_inv)%r
        m=s+(t*n)
        u=m//r
        c_bar=u if u<n else u-n
        c=(c_bar*r_inv) % n
        return c
 

    @staticmethod
    def montgomery_power(base,exponent,n):
        if n == 1:
            return 0
        result = 1
        base = base % n
        while exponent > 0:
            if exponent % 2 == 1:
                result = Montgomery.montgomery_multiply(result , base , n)
            exponent = exponent >> 1
            base = Montgomery.montgomery_multiply(base , base,n )
        return result


In [34]:
class RSA:
    def __init__(self, a: int, b: int):
        # On verifie que a et b sont positifs et que a < b
        if a < 0 or b < 0:
            raise ValueError("a and b must be positive")
        if a >= b:
            raise ValueError("b must be greater than a")

        # On genere deux nombres premiers p et q entre [a, b]
        self.p = sympy.randprime(a, b)
        self.q = sympy.randprime(a, b)

        # On calcule n et phi
        self.n = self.p * self.q
        self.phi = (self.p - 1) * (self.q - 1)

        # On genere un nombre e tel que pgcd(e, phi) = 1 et 1 < e < phi
        self.e = sympy.randprime(1, self.phi)

        # On calcule d tel que e * d = 1 (mod phi)
        self.d = sympy.mod_inverse(self.e, self.phi)

        # r est une puissance de 2 tel que r > n
        self.find_r()

        # r_inv est l'inverse de r modulo n (r * r_inv = 1 (mod n))
        self.r_inv = sympy.mod_inverse(self.r, self.n)

        # n_inv est l'inverse de n modulo r (n * n_inv = -1 (mod r))
        self.n_inv = -sympy.mod_inverse(self.n, self.r) % self.r

    def find_r(self):
        self.r = 1
        while self.r < self.n:
            self.r <<= 1

    def montgomery_mul(self, a: int, b: int) -> int:
        # On transforme a et b en forme Montgomery
        A = a * self.r % self.n
        B = b * self.r % self.n

        s = A * B
        t = (s * self.n_inv) % self.r
        m = s + t * self.n
        u = m // self.r

        if u >= self.n:
            u = u - self.n
            
        result = u * self.r_inv % self.n
        return result

    def montgomery_pow(self, base: int, exponent: int) -> int:
        exponent_binary = [int(x) for x in bin(exponent)[2:]]
        result = base

        # On calcule la puissance de base en utilisant l'exponentiation rapide
        for i in range(1, len(exponent_binary)):
            # On calcule result^2 mod n
            result = result * result % self.n

            if exponent_binary[i] == 1:
                # Si le bit est a 1, on calcule result * base mod n
                result = result * base % self.n

        return result

    def encrypt(self, plaintext):
        # On a C = M^e mod N

        #return self.montgomery_pow(plaintext, self.e)
        return pow(int(plaintext), self.e, self.n)


    def decrypt(self, ciphertext):
        # On a M = C^d mod N

        #return self.montgomery_pow(ciphertext, self.d)
        return pow(int(ciphertext), self.d, self.n)
     



In [35]:


def montgomery_mul(a : int, b :int, n:int):
    
    r = 1
    while r < n:
            r <<= 1
    print(f"On trouve r qui est une puissance de 2 tel que r > n\n")
    print(f"  r = {r}\n")        
    r_inv = sympy.mod_inverse(r, n)

    n_inv = -sympy.mod_inverse(n, r) % r
    

    A = a * r % n
    B = b * r % n
    print("On transforme a et b en forme Montgomery\n")
    print(f"  A = a * r mod n = {a} * {r} mod {n} = {A}")
    print(f"  B = b * r mod n = {b} * {r} mod {n} = {B}\n")
    s = A * B
    print("On calcule s = A * B\n")
    print(f"  s = A * B = {A} * {B} = {s}\n")
    t = (s * n_inv) % r
    print("On calcule t = s * n_inv mod r\n")
    print(f"  t = s * n_inv mod r = {s} * {n_inv} mod {r} = {t}\n")
    m = s + t * n
    print("On calcule m = s + t * n\n")
    print(f"  m = s + t * n = {s} + {t} * {n} = {m}\n")
    
    u = m // r
    print("On divise par r pour obtenir u = m // r\n")
    print(f"  u = m // r = {m} // {r} = {u}\n")
    if u >= n:
        print(f"  u >= n donc u = u - n")
        u = u - n
        print(f"  u = u - n = {u} - {n} = {u}\n")
    
    
    print(f"result = u * r_inv mod n = {u} * {r_inv} mod {n} = {u * r_inv % n}")
    
    return None


a = 12
b = 7
n = 13

print(f"Montgomery mul : {a} * {b} mod {n}\n")
montgomery_mul(a, b, n)


Montgomery mul : 12 * 7 mod 13

On trouve r qui est une puissance de 2 tel que r > n

  r = 16

On transforme a et b en forme Montgomery

  A = a * r mod n = 12 * 16 mod 13 = 10
  B = b * r mod n = 7 * 16 mod 13 = 8

On calcule s = A * B

  s = A * B = 10 * 8 = 80

On calcule t = s * n_inv mod r

  t = s * n_inv mod r = 80 * 11 mod 16 = 0

On calcule m = s + t * n

  m = s + t * n = 80 + 0 * 13 = 80

On divise par r pour obtenir u = m // r

  u = m // r = 80 // 16 = 5

result = u * r_inv mod n = 5 * 9 mod 13 = 6


In [36]:
def rsa_crypt(operation, message):

    global montgomery
    global ori
    montgomery = Montgomery(message,rsa.e,rsa.n)
    if operation == "encrypt":  
        ori = message
        result = Montgomery.montgomery_power(message,rsa.e,rsa.n)
        
    elif operation == "decrypt":
        result = Montgomery.montgomery_power(message,rsa.d,rsa.n)
        result = ori
    return int(message), int(result)


def rsa_set_parameters(a, b):

    global rsa
    rsa = RSA(a, b)
    
    return f"p = {rsa.p} q = {rsa.q}\nn = {rsa.n} phi = {rsa.phi}\ne = {rsa.e} d = {rsa.d}\nr = {rsa.r} r_inv = {rsa.r_inv} n_inv = {rsa.n_inv}"




interface1 = gr.Interface(
    fn=rsa_set_parameters,
    inputs=[
        gr.Number(label="Le debut de l'intervalle"),
        gr.Number(label="La fin de l'intervalle"),
    ],
    title="generateur de clef RSA",
    outputs="text",
)


interface2 = gr.Interface(
    fn=rsa_crypt,
    inputs=[
        gr.Radio(choices=["encrypt", "decrypt"]),
        gr.Number(label="Message"),
    ],
    title="chiffrement et dechifferement RSA integrant la multiplication de Montgomery",
    outputs=[gr.Text(label="input"), gr.Text(label="resultat")],
)

interface1.launch(share=False)
interface2.launch(share=False)

Running on local URL:  http://127.0.0.1:7882

To create a public link, set `share=True` in `launch()`.


Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\xeno\scoop\apps\python311\current\Lib\site-packages\gradio\queueing.py", line 456, in call_prediction
    output = await route_utils.call_process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\xeno\scoop\apps\python311\current\Lib\site-packages\gradio\route_utils.py", line 232, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\xeno\scoop\apps\python311\current\Lib\site-packages\gradio\blocks.py", line 1522, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\xeno\scoop\apps\python311\current\Lib\site-packages\gradio\blocks.py", line 1144, in call_function
    prediction = await anyio.to_thread.run_sync(
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\xeno\scoop\apps\python311\current\Lib\site-packages\anyio\to_thread.py", line 33, in run_sync
    return